##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Распределенное обучение с использованием Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/distribute/keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Содержание

`tf.distribute.Strategy` предоставляет API для распределенного обучения на нескольких устройствах. Цель этого API в том, чтобы позволить пользователям включать распределенное обучение с использованием уже существующих моделей и кода с минимальными изменениями.

В этом руководстве используется `tf.distribute.MirroredStrategy`, которая выполняет внутриграфовую(in-graph) репликацию с синхронным обучением на нескольких графических процессорах на одной машине.
По сути, стратегия копирует все переменные модели на каждый процессор, затем использует технологию [all-reduce](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) для объединения градиентов со всех процессоров и применяет объединенное значение ко всем копиям модели.

`MirroredStrategy` - одна из нескольких распределенных стратегий, доступных в ядре TensorFlow. Вы можете прочитать о других стратегиях в [руководстве по стратегии распространения](../../guide/distribution_training.ipynb).

### Keras API

В этом примере API `tf.keras` используется для построения модели и цикла обучения. Информацию о пользовательских циклах обучения см. в рководстве [использование tf.distribute.Strategy в цикле обучения](training_loops.ipynb).

## Импорт

In [ ]:
# Импорт TensorFlow и TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf

import os

In [ ]:
print(tf.__version__)

## Загрузка датасета

Загрузите датасет MNIST из [TensorFlow Datasets](https://www.tensorflow.org/datasets). Вы получите набор данных в формате `tf.data`.

Установка параметра `with_info` в значение `True` включает получение метаданных датасета, которые в нашем примере сохраняются в переменную `info`.
Помимо прочего, этот объект метаданных включает количество обучающих и тестовых примеров.

In [ ]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

## Определение стратегии распределенного обучения

Создайте объект `MirroredStrategy`. Он будет заниматься распределением и предоставит диспетчер контекста (`tf.distribute.MirroredStrategy.scope`) для построения вашей модели внутри **распределенного** контекста.

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print('Количество устройств: {}'.format(strategy.num_replicas_in_sync))

## Настройка входного конвейера

При обучении модели с несколькими графическими процессорами вы можете эффективно использовать дополнительную вычислительную мощность, увеличив размер пакета. Как правило, используется размер пакета, равный объему памяти графического процессора.

In [ ]:
# Вы также можете выполнить info.splits.total_num_examples
# чтоб получить общее количество примеров в датасете.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

Значение пикселей из диапазона 0-255 [должны быть нормализованы до диапазона 0-1](https://en.wikipedia.org/wiki/Feature_scaling). Определите нормализацию внутри функции.

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

Примените эту функцию к тренировочным и валидационным данным, перемешайте тренировочные данные и [разделите на пакеты для обучения](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch). Обратите внимание, что для повышения производительности мы также кешируем данные.

In [ ]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## Создание модели

Создайте и скомпилируйте модель Keras внутри контекста `strategy.scope`.

In [ ]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

## Определение колбеков


Здесь используются следующие функции обратного вызова:

* *TensorBoard*: этот колбек записывает журнал для TensorBoard, который позже позволяет вам визуализировать графики.
* *Model Checkpoint*: этот колбек сохраняет модель после каждой эпохи.
* *Learning Rate Scheduler*: Используя этот колбек, вы можете запланировать изменение скорости обучения после каждой эпохи/пакета.

Для наглядности добавьте обратный вызов print для отображения *скорости обучения*.

In [ ]:
# Определите директорию для сохранения чекпойнтов

checkpoint_dir = './training_checkpoints'
# Имя файлов
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [ ]:
# Функция снижения скорости обучения.
# Вы можете написать любую подходящую вам функцию.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [ ]:
# Колбек, выводящий скорость обучения в конце каждой эпохи.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

## Обучение и оценка

Теперь обучите модель обычным способом, вызывая `fit` и передавая набор данных, созданный в начале руководства. Этот шаг одинаков и для обычного и для распредленного обучения.

In [ ]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Проверим сохранение контрольных точек.

In [ ]:
!ls {checkpoint_dir}

Чтобы оценить модель, загрузите последнюю контрольную точку и вызовите `evaluate` для проверочных данных.

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

Чтобы увидеть результат, вы можете загрузить и просмотреть логи TensorBoard в терминале.

```
$ tensorboard --logdir=path/to/log-directory
```

In [ ]:
!ls -sh ./logs

## Експорт в SavedModel

Экспортируйте граф и переменные в платформо-независимый формат SavedModel. После сохранения модели вы можете загрузить ее с `strategy.scope` или без него.

In [ ]:
path = 'saved_model/'

In [ ]:
model.save(path, save_format='tf')

Загрузка модели без `strategy.scope`.

In [ ]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

Загрузка модели с `strategy.scope`.

In [ ]:
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

### Примеры и руководства
Вот несколько примеров использования распределенной стратегии с keras fit/compile:
1. [Transformer](https://github.com/tensorflow/models/blob/master/official/nlp/transformer/transformer_main.py), модель, обученная с использованием `tf.distribute.MirroredStrategy`
2. [NCF](https://github.com/tensorflow/models/blob/master/official/recommendation/ncf_keras_main.py), модель, обученная с использованием `tf.distribute.MirroredStrategy`.

Дополнительные примеры приведены в [Руководстве по распределенному обучению](../../guide/distribution_training.ipynb#examples_and_tutorials)

## Что дальше

* Прочтите [руководство по распределенной стратегии](../../guide/distribution_training.ipynb).
* Прочтите руководство [Распределенное обучение с пользовательскими циклами обучения](training_loops.ipynb).
* Посетите [Раздел производительности](../../guide/function.ipynb), чтобы узнать больше о других стратегиях и [инструментах](../../guide/profiler.md), которые можно использовать для оптимизации ваших моделей TensorFlow.

Примечание: `tf.distribute.Strategy` активно разрабатывается, и в ближайшем время мы добавим больше примеров и руководств. Пробуйте и пишите ваши отзывы через [вопросы на GitHub](https://github.com/tensorflow/tensorflow/issues/new).